# Group 27

# Dataset-1 

fetch_20newgroups


multiclassification

# Import Packages 

In [1]:
import numpy as np
#To load the datasets
import os
#to remove puntuation 
import string
#To preprocess the datasets
from nltk.corpus import stopwords #(tokonization)
from sklearn.feature_extraction.text import CountVectorizer
import re, string
#packages for model selection (SVM)
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import metrics
from scipy import sparse
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
#GridSearh
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
#Time
import time

# Upload the Data

In [2]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', remove = ('headers','footers','quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove = ('headers','footers','quotes'))
#list(newsgroups.data)[1:10]

In [3]:
#the target (the output and how many classes do we have)
newsgroups_train.target_names
newsgroups_test.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

# Data Spliting to (Training and Validatoin)

In [4]:
#Data Spliting randomly
## Simple way to do split would be to use scikit-learn's `train_test_split` method
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(newsgroups_train.data, newsgroups_train.target, train_size=0.8, test_size=0.2)
print(len(X_train))
print(len(X_valid))
y_test = newsgroups_test.target
X_test = newsgroups_test.data
print(y_test.shape)
# train tesprint(len(y_train))
# print(y_train[0])
# print(X_train[1])
print(y_test)
print(y_valid)

9051
2263
(7532,)
[ 7  5  0 ...  9  6 15]
[ 4  2 10 ...  6  9 17]


# Preprocessing

In [5]:
def preprocessing(file):
    text_content = []
    exclude = string.punctuation
    exclude = exclude.replace("-", "")
    pattern = r"[{}]".format(exclude)

    for data in file :
        text = re.sub(r"(<br\s*/><br\s*/>)", " ", str(data))
        text = re.sub(pattern, "", str(text))
        text_content.append(text.lower())
    return text_content

In [6]:
#preprocessing
X_train1 = preprocessing(X_train)
y_train = preprocessing(y_train)
X_valid1 = preprocessing(X_valid)
y_valid = preprocessing(y_valid)
X_test1 = preprocessing(X_test)
y_test = preprocessing(y_test)

# Vectorization (count)

In [8]:
#TFID
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer()
Train_set = tf_idf_vectorizer.fit_transform(X_train1)
Valid_set = tf_idf_vectorizer.transform(X_valid1)
Test_set = tf_idf_vectorizer.transform(X_test1)

# GridSearch and Model Selection

#  SVM

In [24]:
#Define the hyper-parameters
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    t0=time.time()
    clf.fit(Train_set, y_train)
    print ("training time:", round(time.time()-t0, 3), "s")
    
    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    t1=time.time()
    y_pred = clf.predict(Valid_set)
    print ("predict time:", round(time.time()-t1, 3), "s")
    print(metrics.classification_report(y_valid, y_pred, digits = 5))


# Tuning hyper-parameters for precision



C:\Users\zalmhe\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\zalmhe\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


training time: 1857.495 s
Best parameters set found on development set:

{'C': 10, 'kernel': 'linear'}

Grid scores on development set:

0.003 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.003 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.003 (+/-0.000) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.003 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.682 (+/-0.023) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.003 (+/-0.000) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.715 (+/-0.002) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.683 (+/-0.023) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.711 (+/-0.011) for {'C': 1, 'kernel': 'linear'}
0.722 (+/-0.002) for {'C': 10, 'kernel': 'linear'}
0.706 (+/-0.012) for {'C': 100, 'kernel': 'linear'}
0.705 (+/-0.012) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.



C:\Users\zalmhe\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


training time: 1851.186 s
Best parameters set found on development set:

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.050 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.050 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.050 (+/-0.000) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.050 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.520 (+/-0.025) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.050 (+/-0.000) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.690 (+/-0.007) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.520 (+/-0.025) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.685 (+/-0.007) for {'C': 1, 'kernel': 'linear'}
0.641 (+/-0.006) for {'C': 10, 'kernel': 'linear'}
0.598 (+/-0.004) for {'C': 100, 'kernel': 'linear'}
0.598 (+/-0.003) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full ev

# Testing the Final (Selected) Model 1 for SVM

In [16]:
#####Optimal Solution 1##############
#kernel is linear, C=10.

############ Valid+Train ##################
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn import metrics
import time

t0=time.time()
clf = svm.SVC(kernel='linear',C = 10).fit(Train_set, y_train)
print ("training time:", round(time.time()-t0, 3), "s")
y_pred = clf.predict(Valid_set)
print(metrics.classification_report(y_valid, y_pred, digits = 5))

############ Test #################
t2=time.time()
y_pred = clf.predict(Test_set)
print ("predict time for test:", round(time.time()-t2, 3), "s")
print(metrics.classification_report(y_test, y_pred, digits = 5))

training time: 75.769 s
              precision    recall  f1-score   support

           0    0.59794   0.63043   0.61376        92
           1    0.69643   0.69643   0.69643       112
          10    0.96078   0.80992   0.87892       121
          11    0.91089   0.69697   0.78970       132
          12    0.69421   0.62687   0.65882       134
          13    0.86869   0.71074   0.78182       121
          14    0.78070   0.74790   0.76395       119
          15    0.67544   0.63115   0.65254       122
          16    0.77451   0.73832   0.75598       107
          17    0.82727   0.79130   0.80889       115
          18    0.64286   0.59341   0.61714        91
          19    0.56000   0.32558   0.41176        86
           2    0.66667   0.67327   0.66995       101
           3    0.65909   0.58000   0.61702       100
           4    0.72917   0.64220   0.68293       109
           5    0.79200   0.75573   0.77344       131
           6    0.82828   0.66129   0.73543       124
   

# Testing the Final (Selected) Model 2 for SVM

In [17]:
#####Optimal Solution 2##############
#kernel is radical basis function
#C= 1000
#gamma = 0.001
############ Valid ##################
from sklearn import svm
from sklearn import metrics
import time

t0=time.time()
clf = svm.SVC(kernel='rbf',C = 1000, gamma = 0.001 ).fit(Train_set, y_train)
print ("training time:", round(time.time()-t0, 3), "s")

y_pred = clf.predict(Valid_set)
print(metrics.classification_report(y_valid, y_pred, digits = 5))
t2=time.time()
y_pred = clf.predict(Test_set)
print ("predict time for test:", round(time.time()-t2, 3), "s")
print(metrics.classification_report(y_test, y_pred, digits = 5))


training time: 78.753 s
              precision    recall  f1-score   support

           0    0.59223   0.66304   0.62564        92
           1    0.65600   0.73214   0.69198       112
          10    0.93519   0.83471   0.88210       121
          11    0.88182   0.73485   0.80165       132
          12    0.69841   0.65672   0.67692       134
          13    0.85321   0.76860   0.80870       121
          14    0.76667   0.77311   0.76987       119
          15    0.67742   0.68852   0.68293       122
          16    0.76190   0.74766   0.75472       107
          17    0.79825   0.79130   0.79476       115
          18    0.68235   0.63736   0.65909        91
          19    0.58537   0.27907   0.37795        86
           2    0.68182   0.74257   0.71090       101
           3    0.72093   0.62000   0.66667       100
           4    0.70536   0.72477   0.71493       109
           5    0.87611   0.75573   0.81148       131
           6    0.81982   0.73387   0.77447       124
   

# Decision Tree GridSearch

In [40]:
def Model_Selection_train(train_inp, y_tra, valid_inp, y_vald, clf, params):
#def Model_Selection_train(train_inp, y_tra, valid_inp, y_vald, test_inp, y_tst, clf, params):
    train_input = train_inp
    valid_input = valid_inp
    #test_input  = test_inp
    
    train_label = y_tra
    valid_label = y_vald
    #test_label  = y_tst
    
    if params != None :

        combine_input = sparse.vstack([train_input, valid_input])
        combine_label = np.concatenate((train_label, valid_label))
        fold = [-1 for i in range(train_input.shape[0])] + [0 for i in range(valid_input.shape[0])]
        #fold = 5
        ps = PredefinedSplit(test_fold = fold)
        clf = GridSearchCV(clf, params, cv=ps, refit=True)
        clf.fit(combine_input, combine_label)
    else:
        clf.fit(train_input, train_label)
        
    best_param = None if params==None else clf.best_params_

    f1_train = f1_score(train_label, clf.predict(train_input), average = 'micro')
    f1_valid = f1_score(valid_label, clf.predict(valid_input), average = 'micro')
    #f1_test = f1_score(test_label, clf.predict(test_input), average = 'micro')
    Acc_train = accuracy_score(train_label, clf.predict(train_input), normalize = True)
    Acc_valid = accuracy_score(valid_label, clf.predict(valid_input), normalize = True)
    #Acc_test = accuracy_score(test_label, clf.predict(test_input), normalize = True)

    return f1_train, f1_valid, Acc_train, Acc_valid, best_param

In [20]:
#fold is predefined
#param = [{'criterion': ['gini'], 'splitter': ['best'] , 'min_samples_leaf': [np.arange(1, 20)], 'max_features': ['sqrt'], 'max_leaf_nodes': [1000 * i for i in range(3, 6)]}]
#Decision Tree
param = [{'max_features': [10000 * i for i in range(2, 9)], 'max_leaf_nodes': [1000 * i for i in range(1, 10)], 'min_samples_leaf': np.arange(1, 20)}]
#param = [{'max_depth': [i for i in range(150, 200)], 'max_features': [10000 * i for i in range(1, 9)], 'max_leaf_nodes': [10000 * i for i in range(1, 10)], 'min_samples_leaf': np.arange(1, 20)}]
t1=time.time()
pred = Model_Selection_train(Train_set, y_train, Valid_set, y_valid, Test_set, y_test, DecisionTreeClassifier(), param)
print(set, "Decision Tree \n(train, valid, test) = ", pred[:7])
print("best params = {}\n".format(pred[6]))
print ("predict time:", round(time.time()-t1, 3), "s")

#param = [{'max_depth': [i for i in range(15, 20)], 'max_features': [10000 * i for i in range(2, 9)], 'max_leaf_nodes': [1000 * i for i in range(1, 10)], 'min_samples_leaf': np.arange(1, 20)}]



<class 'set'> Decision Tree 
(train, valid, test) =  (0.5488896254557507, 0.5475033141847105, 0.3777217206585236, 0.5488896254557507, 0.5475033141847105, 0.3777217206585236, {'max_features': 20000, 'max_leaf_nodes': 5000, 'min_samples_leaf': 13})
best params = {'max_features': 20000, 'max_leaf_nodes': 5000, 'min_samples_leaf': 13}

predict time: 8279.223 s


In [41]:
#fold is 5
#Run it with no maximum depth (now im doing this)
#param = [{'criterion': ['gini'], 'splitter': ['best'] , 'min_samples_leaf': [np.arange(1, 20)], 'max_features': ['sqrt'], 'max_leaf_nodes': [1000 * i for i in range(3, 6)]}]
#Decision Tree
param = [{'max_features': [10000 * i for i in range(2, 9)], 'max_leaf_nodes': [1000 * i for i in range(1, 10)], 'min_samples_leaf': np.arange(1, 20)}]
#param = [{'max_depth': [i for i in range(150, 200)], 'max_features': [10000 * i for i in range(1, 9)], 'max_leaf_nodes': [10000 * i for i in range(1, 10)], 'min_samples_leaf': np.arange(1, 20)}]
t1=time.time()
pred = Model_Selection_train(Train_set, y_train, Valid_set,y_valid, DecisionTreeClassifier(), param)
print(set, "Decision Tree \n(train, valid, test) = ", pred[:4])
print("best params = {}\n".format(pred[4]))
print ("predict time:", round(time.time()-t1, 3), "s")

<class 'set'> Decision Tree 
(train, valid, test) =  (0.9741465031488233, 0.9765797613787008, 0.9741465031488233, 0.9765797613787008)
best params = {'max_features': 60000, 'max_leaf_nodes': 9000, 'min_samples_leaf': 1}

predict time: 8389.561 s


# Testing the Final (selected) model for Decision Tree

In [18]:
############Valid####################
t0=time.time()
clf = DecisionTreeClassifier(max_features= 60000, max_leaf_nodes = 9000, min_samples_leaf = 1).fit(Train_set, y_train)
print ("training time:", round(time.time()-t0, 3), "s")
t1=time.time()
y_pred = clf.predict(Valid_set)
print ("predict time:", round(time.time()-t1, 3), "s")
print(metrics.classification_report(y_valid, y_pred, digits = 5))
############ Test #################
t2=time.time()
y_pred = clf.predict(Test_set)
print ("predict time for test:", round(time.time()-t2, 3), "s")
print(metrics.classification_report(y_test, y_pred, digits = 5))


training time: 23.462 s
predict time: 0.003 s
              precision    recall  f1-score   support

           0    0.25287   0.23913   0.24581        92
           1    0.38462   0.35714   0.37037       112
          10    0.61345   0.60331   0.60833       121
          11    0.70000   0.53030   0.60345       132
          12    0.32174   0.27612   0.29719       134
          13    0.42029   0.47934   0.44788       121
          14    0.45690   0.44538   0.45106       119
          15    0.44954   0.40164   0.42424       122
          16    0.35455   0.36449   0.35945       107
          17    0.61386   0.53913   0.57407       115
          18    0.27174   0.27473   0.27322        91
          19    0.29412   0.17442   0.21898        86
           2    0.39837   0.48515   0.43750       101
           3    0.24138   0.21000   0.22460       100
           4    0.41406   0.48624   0.44726       109
           5    0.55469   0.54198   0.54826       131
           6    0.45000   0.43548  